In [77]:
!pip install neattext
import pandas as pd
import re
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import neattext as nt
import neattext.functions as nfx
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import train_test_split


In [78]:
df = pd.read_csv('../input/twitter-airline-sentiment/Tweets.csv')

In [79]:
df.head(10)

In [80]:
!pip install wordcloud

### WordCloud to visualize Frequency Of words

In [81]:
def frequency(data):
    tweet_data = []
    for words in data.split():
        tweet_data.extend(words)
    freq = pd.Series(tweet_data).value_counts()
    return freq[:40]

In [82]:

word_freq = frequency(df.text.str)
wc = WordCloud(width=400,height=330,max_words=100,background_color='black').generate_from_frequencies(word_freq)
plt.figure(figsize=(12,12))
plt.imshow(wc,interpolation='bilinear')
plt.axis('off')
plt.show()

In [83]:

#utput=[]
#or i in tqdm(X):
    #ew_text = re.sub('[^a-zA-Z]',' ',str(i))
    #ew_text = re.sub(r'[@&!]',' ',new_text)
    #ew_text = new_text.lower()
    #ew_text = new_text.split()
    #ew_text = [lm.lemmatize(j) for j in new_text if j not in set(stopwords.words('english'))]
    #utput.append(' '.join(str(k) for k in new_text))


# Text Preprocessing Using Neattext

In [84]:
!pip install neattext

In [85]:
df = df[['text','airline_sentiment']]

In [86]:
df

### Before Cleaning Our data Contains Noise that can be checked by noise_scan()

In [87]:
df['text'].apply(lambda x:nt.TextFrame(x).noise_scan()['text_noise'])

### <font color = 'Blue'> Remove Userhandles present in our dataset using remove_userhandles

In [88]:
df['clean_text'] = df['text'].apply(nfx.remove_userhandles)

In [89]:
df['clean_text']

### <font color = 'Blue'> Remove Hashtags

In [90]:
df['clean_text'] = df['clean_text'].apply(nfx.remove_hashtags)

### <font color='Blue'> To lower Down the data

In [91]:
df['clean_text'] = df['clean_text'].apply(lambda x: x.lower())

### <font color='Blue'> To remove Patterns/special characters in our data

In [92]:
df['clean_text'] = df['clean_text'].apply(lambda x: nfx.remove_custom_pattern(x,term_pattern=r'&#$\S+'))

In [93]:
df['clean_text'] = df['clean_text'].apply(lambda x: nfx.remove_custom_pattern(x,term_pattern=r'&#$\S+'))

In [94]:
df['clean_text'] = df['clean_text'].apply(lambda x: nfx.remove_custom_pattern(x,term_pattern=r'RT'))

In [95]:
!pip install contractions

### <font color='Blue'> Contradictions For converting Apostrophic(') into original like you've--> you have

In [96]:
import contractions
df['clean_text'] = df['clean_text'].apply(lambda x:contractions.fix(x))

### <font color='Blue'> To remove extra special characters like @$#!!

In [97]:
df['clean_text'] = df['clean_text'].apply(nfx.remove_special_characters)

### <font color='Blue'> To remove Multiple Spaces

In [98]:
df['clean_text'] = df['clean_text'].apply(nfx.remove_multiple_spaces)

### <font color='Blue'> To remove Stopwords

In [99]:
df['clean_text'] = df['clean_text'].apply(lambda x: nt.TextFrame(x).remove_stopwords())

In [100]:
df['clean_text']

### It's Showing that our Text data is clean doesn't contain any noise now

In [101]:
df['clean_text'].apply(lambda x:x.noise_scan()['text_noise'])

In [102]:
df

In [103]:
X = df['clean_text']
Y = df['airline_sentiment']
Y = Y.map({'positive':2,'neutral':1,'negative':0})

### Converted Series Data Into list of sentences for BagofWords Model

In [104]:
Air_data = []
for i in range(len(X)):
    y = X[i]
    Air_data.append(y.text)
    

In [105]:
len(Air_data)

In [106]:
!pip install sklearn

### convert our data into array of sentences to feed as input to Word2Vec

In [107]:
vec_of_sent = []
for i in Air_data:
    z = list(i.split(" "))
    vec_of_sent.append(z)

In [108]:
from gensim.models import Word2Vec
model = Word2Vec(vec_of_sent, min_count=1,vector_size=300,workers=5)

In [109]:
model.epochs

In [ ]:
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")
model.train(vec_of_sent,total_examples = len(vec_of_sent),epochs=model.epochs)
vocab = list(model.wv.key_to_index)
vocab

In [111]:
len(Y)

In [112]:
#x = list(model.wv.key_to_index)
similar_words = model.wv.most_similar('thanks')
print(similar_words)

### PCA to visualize which model will best fit to our data

In [113]:
'''
import seaborn as sb
import gc
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
gc.collect()
pca = PCA(n_components=2)
pca_d = pca.fit_transform(Air_fit.toarray())
labels = Y.tolist()
plt.figure(figsize=(20,10))
plt.scatter(pca_d[:,0],pca_d[:,1],c = labels,cmap='plasma')
plt.xlabel("First Principle component")
plt.ylabel("Second principle component")
'''


### CountVectorization(Bag of Words) to convert word into one-hot Encoded form.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer(ngram_range=(2,2))
Air_fit = count_vec.fit_transform(Air_data)
print(count_vec.get_feature_names())

2 classes have less value_counts so to overcome bias issue,oversampling need to use first before splitting

In [115]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(Air_fit,Y)

In [116]:
x_train,x_test,y_train,y_test = train_test_split(X_sm,y_sm,random_state = 1,test_size = 0.3)

In [117]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report

### <font color='purple' > Random Forest bagging model

In [118]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train,y_train)
train_pred = random_forest.predict(x_train)
test_pred = random_forest.predict(x_test)
train_matrix = accuracy_score(y_train,train_pred)
test_matrix = accuracy_score(y_test,test_pred)
print(classification_report(y_test,test_pred))
print(train_matrix)
print(test_matrix)


In [119]:
import pickle
forest_model ='./Airline_twitter_nlp.pickle'

In [120]:
with open(forest_model,'wb') as f:
    pickle.dump(random_forest,f)

### <font color='purple'> Support Vector classifier

In [121]:
from sklearn.svm import SVC
svc = SVC(kernel='sigmoid')
svc.fit(x_train,y_train)
train_pred = svc.predict(x_train)
train_accuracy = accuracy_score(y_train,train_pred)
test_pred = svc.predict(x_test)
test_accuracy = accuracy_score(y_test,test_pred)
print(train_accuracy)
print(test_accuracy)


### <font color = 'purple'> Multinomial Naive Bayes

In [122]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(x_train,y_train)
pred = naive_bayes.predict(x_train)
print(accuracy_score(y_train,pred))
test_pred = naive_bayes.predict(x_test)
print(accuracy_score(y_test,test_pred))


### <font color='purple'> Neural Network

In [123]:
tweet_data = pd.Series(vec_of_sent)
vocabulary = set()
for tweet in tweet_data:
    for word in tweet:
        if word not in vocabulary:
            vocabulary.add(word)
vocab_length = len(vocabulary)
max_length_seq = 0
for tweet in tweet_data:
    if len(tweet)>max_length_seq:
        max_length_seq = len(tweet)
        
print("Vocab_length",vocab_length)
print("Max_seq_length",max_length_seq)

    

### To convert words into sequence of numbers

In [124]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [125]:
tokenizer = Tokenizer(num_words = vocab_length)
tokenizer.fit_on_texts(tweet_data)
sequences = tokenizer.texts_to_sequences(tweet_data)
word_index = tokenizer.word_index
model_inputs = pad_sequences(sequences,maxlen = max_length_seq,padding='post')

In [126]:
df

In [ ]:
sequences

In [ ]:
word_index

In [129]:
model_inputs

In [130]:
labels = np.array(Y)

In [176]:
X_train,X_test,Y_train,Y_test = train_test_split(model_inputs,labels,test_size = 0.2,random_state=1,stratify=labels)

### Training

In [177]:
import tensorflow as tf
import tensorflow.keras as keras

In [178]:
embedding_dim=32
input = tf.keras.Input(shape=(max_length_seq,))
embedding = tf.keras.layers.Embedding(input_dim = vocab_length,output_dim =embedding_dim,input_length = max_length_seq)(input)
flatten = tf.keras.layers.Flatten()(embedding)        #to flatten the embedding
gru = tf.keras.layers.GRU(units=embedding_dim)(embedding)
flatten_gru = tf.keras.layers.Flatten()(gru)             #to flatten the gru output
comb = tf.keras.layers.concatenate([flatten,flatten_gru])   #merge them to feed into dense layer
output = tf.keras.layers.Dense(3,activation='softmax')(comb) 
model = tf.keras.Model(input,output)
tf.keras.utils.plot_model(model)

In [179]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

r = model.fit(X_train,Y_train,validation_split=0.5,batch_size=32,epochs = 10)

In [158]:
r.history['loss']

In [159]:
model.evaluate(X_test,Y_test)

1. **Accuracy for Random Forest         = 98%**
2. **Accuracy for MultinomialNaiveBayes = 75%**
3. **Accuracy for SVM                   = 65%**
4. **Accuracy for neural network        = 98%**